In [61]:
from sklearn.datasets import load_iris
import numpy as np
from sklearn.model_selection import train_test_split
import cvxopt
from sklearn.metrics import accuracy_score

iris = load_iris()
data_set = iris.data
labels = iris.target

train_data_set, test_data_set, train_labels, test_labels = train_test_split(
    data_set, labels, test_size=0.2, random_state=15)

#train_labels[train_labels<1] = -1
#test_labels[test_labels<1] = -1

#objects_count = len(train_labels)

ModuleNotFoundError: No module named 'cvxopt'

In [ ]:
len(train_data_set)

In [ ]:
train_data_set[0:3]

In [ ]:
train_labels[0:3]

# Exercises

In this section we have two exercises:
1. Implement the polynomial kernel.
2. Implement the multiclass C-SVM.

## Polynomial kernel

You need to extend the ``build_kernel`` function and implement the polynomial kernel if the ``kernel_type`` is set to 'poly'. The equation that needs to be implemented:
\begin{equation}
K=(X^{T}*Y)^{d}.
\end{equation}

In [ ]:
def build_kernel(data_set, kernel_type='linear', d_poly_exp = 1):
    kernel = np.dot(data_set, data_set.T)
    if kernel_type == 'rbf':
        sigma = 1.0
        objects_count = len(data_set)
        b = np.ones((len(data_set), 1))
        kernel -= 0.5 * (np.dot((np.diag(kernel)*np.ones((1, objects_count))).T, b.T)
                         + np.dot(b, (np.diag(kernel) * np.ones((1, objects_count))).T.T))
        kernel = np.exp(kernel / (2. * sigma ** 2))
    elif kernel_type == 'poly':
        # funkcja z wykładu: polynomial K (x i , x j ) = ( x ik x jk + a)**d
        # zakłdam, powyższy wzór jest po prostu wielomianowym jądrem JEDNORODNYM
        # funkcja z wykłady jest poprawna, więc z niej skorzystam, ta nie
        kernel = np.power(np.dot(data_set, data_set.T), d_poly_exp)
        
    return kernel

## Implement a multiclass C-SVM

Use the classification method that we used in notebook 7.3 and IRIS dataset to build a multiclass C-SVM classifier. Most implementation is about a function that will return the proper data set that need to be used for the prediction. You need to implement:
- ``choose_set_for_label``
- ``get_labels_count``

In [ ]:
# Kernel'e wprowadzmy po to aby dane dało się liniowo rozdzielić hiperpłaszczyzną
# W C-SVM chodzi o to, żeby ze zbioru treningowego wybrać odstające obserwacje, które zaburzają klasfikator.
# i w tym przypadku nadal minimalizujemy funkcję marginesu ale skorygowaną o C * sum("slack variables")

# proper data_set that need's to be used for prediction - aha o to tu się chodzi o to, że trzeba tak podzielić 
# zbiór train-test na kombinacje klas po dwie
# Each 2 combination is a vote for a class that a given is classified to. The object is assigned a label that 
# gets most votes.

# nowa strategia: to oznacza, że wypadało by wytrenować n(n-1)/2 klasyfikatorów, tu 6
# potem próbkę testową wpuszczmy w każdy z 6 ciu klasyfikatorów i każdy z nich głosuje do której klasy należy próbka
# albo wytrenuje 3 klasyfikatory 1 i reszta i one będą głosować i większość zdecyduje - - to będzie: one vs all

In [ ]:
# wyrzucenie ze zbioru tych odstających
def choose_set_for_label(data_set, label,labels): # tu podamy train_data_set
    # indexy wektorów rozpinających hiperpłaszczyznę i wartości lambda
    # równanie hiperpłaszczyzny: w⋅x+b=0
    # w - weight vector - wyliczamy ze wzoru w = sum x_i y_i lambda_i 
    # działmy najpierw funkcją train na cały data_set, ponieważ chcemy z niego pózniej wykluczyć outliery
    # dopiero po tym podzielimy na train i test
    labels_local = np.copy(labels)
    labels_local[labels!=label] = -1
    labels_local[labels==label] = 1
    train_data_set, test_data_set, train_labels, test_labels = train_test_split(data_set, labels_local, test_size=0.2, random_state=10)
    
    return train_data_set, test_data_set, train_labels, test_labels
    

In [ ]:
def get_labels_count(labels): # labels insted of dataset, because data was prepared that way earlier in the notebook
    labels_count = {}
    uniq = np.unique(labels)
    for label in uniq:
        labels_count[label] = list(labels).count(label)
    return labels_count

In [ ]:
# żeby wyznaczyć slack variables, to najpierw trzeba mieć hiperpłaszczyznę
# żeby te odległości policzyć
# Use the code that we have implemented earlier:

In [ ]:
def train(train_data_set, train_labels, kernel_type='linear', C=10, threshold=1e-5):
    kernel = build_kernel(train_data_set, kernel_type=kernel_type)

    P = train_labels * train_labels.transpose() * kernel
    q = -np.ones((objects_count, 1))
    
    # W przypadku "hard-margin" SVM podając macirz h do cvxopt nie uwzględniamy C, a dla soft marigin już uwzględnimy
    # Tu już jest uwzględnioe, więc zakadam, tu już jest robione przez "dual"
    # No ok, czyli właściwie minimalizowana jest już ta funkcja co powinna być
    
    # Ta funkcja zwróci nam macirz alf, to mamy zdefiniowaną macierz wektory nośne
    # Wektory nośne definiują hiperpłaszczyznę - f (x) = β 0 + sum(lambda_i*K (x i , x j ) )
    
    G = np.concatenate((np.eye(objects_count), -np.eye(objects_count)))
    h = np.concatenate((C * np.ones((objects_count, 1)), np.zeros((objects_count, 1))))

    A = train_labels.reshape(1, objects_count)
    A = A.astype(float)
    b = 0.0

    sol = cvxopt.solvers.qp(cvxopt.matrix(P), cvxopt.matrix(q), cvxopt.matrix(G), cvxopt.matrix(h), cvxopt.matrix(A), cvxopt.matrix(b))

    lambdas = np.array(sol['x'])

    support_vectors_id = np.where(lambdas > threshold)[0]
    vector_number = len(support_vectors_id)
    support_vectors = train_data_set[support_vectors_id, :]

    lambdas = lambdas[support_vectors_id]
    targets = train_labels[support_vectors_id]

    b = np.sum(targets)
    for n in range(vector_number):
        b -= np.sum(lambdas * targets * np.reshape(kernel[support_vectors_id[n], support_vectors_id], (vector_number, 1)))
    b /= len(lambdas)

    return lambdas, support_vectors, support_vectors_id, b, targets, vector_number

def build_kernel(data_set, kernel_type='linear'):
    kernel = np.dot(data_set, data_set.T)
    if kernel_type == 'rbf':
        sigma = 1.0
        objects_count = len(data_set)
        b = np.ones((len(data_set), 1))
        kernel -= 0.5 * (np.dot((np.diag(kernel)*np.ones((1, objects_count))).T, b.T)
                         + np.dot(b, (np.diag(kernel) * np.ones((1, objects_count))).T.T))
        kernel = np.exp(kernel / (2. * sigma ** 2))
    return kernel

def classify_rbf(test_data_set, train_data_set, lambdas, targets, b, vector_number, support_vectors, support_vectors_id):
    kernel = np.dot(test_data_set, support_vectors.T)
    sigma = 1.0
    #K = np.dot(test_data_set, support_vectors.T)
    #kernel = build_kernel(train_data_set, kernel_type='rbf')
    c = (1. / sigma * np.sum(test_data_set ** 2, axis=1) * np.ones((1, np.shape(test_data_set)[0]))).T
    c = np.dot(c, np.ones((1, np.shape(kernel)[1])))
    #aa = np.dot((np.diag(K)*np.ones((1,len(test_data_set)))).T[support_vectors_id], np.ones((1, np.shape(K)[0]))).T
    sv = (np.diag(np.dot(train_data_set, train_data_set.T))*np.ones((1,len(train_data_set)))).T[support_vectors_id]
    aa = np.dot(sv,np.ones((1,np.shape(kernel)[0]))).T
    
    kernel = kernel - 0.5 * c - 0.5 * aa
    kernel = np.exp(kernel / (2. * sigma ** 2))

    y = np.zeros((np.shape(test_data_set)[0], 1))
    for j in range(np.shape(test_data_set)[0]):
        for i in range(vector_number):
            # równanie definiujące hiperpłaszczyznę
            # kiedy odlgłość elementu od hiperpowierzchni jest większ lub równa 1, to prowadzi on do pogorszenia klasyfikacji
            y[j] += lambdas[i] * targets[i] * kernel[j, i]
        y[j] += b
    return np.sign(y)

In [ ]:
# modification:

train_data_set, test_data_set, train_labels, test_labels = choose_set_for_label(data_set, labels[0], labels)
predicted_one_vs_all = np.empty((0, len(test_labels)))

from sklearn.metrics import accuracy_score
# Chodzi o to, że wcześniej odrzuciliśmy obserwacje o innej klasie niż 0 lub 1 - teraz je uwzględnimy
for label in np.unique(labels):
    train_data_set, test_data_set, train_labels, test_labels = choose_set_for_label(data_set, label, labels)
    lambdas, support_vectors, support_vectors_id, b, targets, vector_number = train(train_data_set, train_labels, kernel_type='rbf')
    predicted = classify_rbf(test_data_set, train_data_set, lambdas, targets, b, vector_number, support_vectors, support_vectors_id)
    predicted = list(predicted.astype(int))
    print(accuracy_score(pred_rbf, test_labels))
    predicted_one_vs_all = np.vstack((predicted_one_vs_all, np.ndarray.flatten(pred_rbf)))

predicted_bis = np.zeros((len(predicted_one_vs_all[0])))
for label_idx, predicted_label_vs_all in enumerate(predicted_one_vs_all):
    label = np.unique(labels)[label_idx]
    predicted_bis[predicted_label_vs_all > -1] = label
    
print("Prediction on test data:", predicted_bis)
print("True test data:", test_labels_all)
print("Accuracy:", accuracy_score(predicted_bis, test_labels_all))
    
#print(accuracy_score(predicted, test_labels))